In [ ]:
import utils
import time
import os
import numpy as np
from os.path import isdir
from sklearn.ensemble import ExtraTreesClassifier

preprocessed_data_path = './preprocessed_data/'
pred_path = './results/'
x_train_r, y_train_r, w_train_r, e_train_r, x_test_r, id_test_r = utils.load_preprocessed_pd_data(preprocessed_data_path)

In [ ]:
def era_k_fold_with_weight(x, y, w, e, n_valid, n_cv):

    n_era = 20
    n_traverse = n_era // n_valid
    n_rest = n_era % n_valid

    if n_cv % n_valid != 0:
        raise ValueError("Number of CV is not an integer multiple of number of validation era!")

    if n_rest != 0:
        n_traverse += 1

    n_epoch = n_cv // n_traverse
    trained_cv = []

    for epoch in range(n_epoch):

        era_idx = []
        era_idx.append(list(range(1, n_era + 1)))

        if n_rest == 0:

            for i in range(n_traverse):

                # Choose eras that have not used
                if trained_cv != []:
                    valid_era = np.random.choice(era_idx[i], n_valid, replace=False)
                    while any(set(valid_era) == i_cv for i_cv in trained_cv):
                        print('trained_cv:', trained_cv)
                        print('valid_era:', valid_era)
                        print('This CV split has been chosen, choosing another one...')
                        if valid_era != era_idx[i]:
                            valid_era = np.random.choice(era_idx[i], n_valid, replace=False)
                        else:
                            valid_era = np.random.choice(range(1, n_era+1), n_valid, replace=False)
                else:
                    valid_era = np.random.choice(era_idx[i], n_valid, replace=False)

                # Generate era set for next choosing
                if i != n_traverse - 1:
                    era_next = [rest for rest in era_idx[i] if rest not in valid_era]
                    era_idx.append(era_next)

                train_index = []
                valid_index = []

                # Generate train-validation split index
                for ii, ele in enumerate(e):

                    if ele in valid_era:
                        valid_index.append(ii)
                    else:
                        train_index.append(ii)

                np.random.shuffle(train_index)
                np.random.shuffle(valid_index)

                # Training data
                x_train = x[train_index]
                y_train = y[train_index]
                w_train = w[train_index]

                # Validation data
                x_valid = x[valid_index]
                y_valid = y[valid_index]
                w_valid = w[valid_index]

                trained_cv.append(set(valid_era))

                yield x_train, y_train, w_train, x_valid, y_valid, w_valid

        # n_cv is not an integer multiple of n_valid
        else:

            for i in range(n_traverse):

                if i != n_traverse - 1:

                    valid_era = []
                    if trained_cv != []:
                        valid_era = np.random.choice(era_idx[i], n_valid, replace=False)
                        while any(set(valid_era) == i_cv for i_cv in trained_cv):
                            print('trained_cv:', trained_cv)
                            print('valid_era:', valid_era)
                            print('This CV split has been chosen, choosing another one...')
                            valid_era = np.random.choice(era_idx[i], n_valid, replace=False)
                    else:
                        valid_era = np.random.choice(era_idx[i], n_valid, replace=False)

                    era_next = [rest for rest in era_idx[i] if rest not in valid_era]
                    era_idx.append(era_next)

                    train_index = []
                    valid_index = []

                    for ii, ele in enumerate(e):

                        if ele in valid_era:
                            valid_index.append(ii)
                        else:
                            train_index.append(ii)

                    np.random.shuffle(train_index)
                    np.random.shuffle(valid_index)

                    # Training data
                    x_train = x[train_index]
                    y_train = y[train_index]
                    w_train = w[train_index]

                    # Validation data
                    x_valid = x[valid_index]
                    y_valid = y[valid_index]
                    w_valid = w[valid_index]

                    trained_cv.append(set(valid_era))

                    yield x_train, y_train, w_train, x_valid, y_valid, w_valid

                else:

                    era_idx_else = [t for t in list(range(1, n_era + 1)) if t not in era_idx[i]]

                    valid_era = []
                    valid_era = era_idx[i] + list(np.random.choice(era_idx_else, n_valid - n_rest, replace=False))
                    while any(set(valid_era) == i_cv for i_cv in trained_cv):
                        print('This CV split has been chosen, choosing another one...')
                        valid_era = era_idx[i] + list(
                            np.random.choice(era_idx_else, n_valid - n_rest, replace=False))

                    train_index = []
                    valid_index = []

                    for ii, ele in enumerate(e):

                        if ele in valid_era:
                            valid_index.append(ii)
                        else:
                            train_index.append(ii)

                    np.random.shuffle(train_index)
                    np.random.shuffle(valid_index)

                    # Training data
                    x_train = x[train_index]
                    y_train = y[train_index]
                    w_train = w[train_index]

                    # Validation data
                    x_valid = x[valid_index]
                    y_valid = y[valid_index]
                    w_valid = w[valid_index]

                    trained_cv.append(set(valid_era))

                    yield x_train, y_train, w_train, x_valid, y_valid, w_valid

In [ ]:
import model

In [ ]:
x = x_train_r
y = y_train_r
w = w_train_r
e = e_train_r
n_valid = 4
n_cv = 1000
for i, _ in enumerate(era_k_fold_with_weight(x, y, w, e, n_valid, n_cv)):
    print(i)